In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%amtplotlib inline 

UsageError: Line magic function `%amtplotlib` not found.


In [3]:
xgb_result_path = 'submit_xgb_probility.csv'
lgb_result_path ="submit_lgb_probility"

In [4]:
oof_xgb= pd.read_csv("train_xgb_result.csv")
oof_lgb = pd.read_csv("train_lgb_result.csv")
predictions_xgb = pd.read_csv("submit_xgb_probility.csv")
predictions_lgb = pd.read_csv("submit_lgb_probility.csv") 

In [9]:
predictions_xgb

,Unnamed: 0,sid,label
0,0,66102,0.218912
1,1,90780,0.568260
2,2,115831,0.069121
3,3,111132,0.069065
4,4,103171,0.896793
...,...,...,...
149995,149995,24950,0.901183
149996,149996,66628,0.908581
149997,149997,20269,0.908655
149998,149998,104797,0.936764


In [10]:
from sklearn import linear_model
from  sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error 


oof_lgb= oof_lgb['label']
oof_xgb= oof_xgb['label']
predictions_lgb= predictions_lgb['label']
predictions_xgb= predictions_xgb['label']

train  =pd.read_csv( 'train_lightgbm.csv' )
X= train.drop('label' ,axis =1 )
Y= train['label']

# 将lgb和xgb和ctb的结果进行stacking
train_stack =  pd.concat( [ pd.Series(oof_lgb ) ,pd.Series( oof_xgb ) ] ,axis =1 ) 
test_stack = pd.concat( [ pd.Series(predictions_lgb) ,
                         pd.Series( predictions_xgb )  ] ,axis =1 ) 


folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=17)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,Y)):
    trn_data, trn_y = train_stack.iloc[ trn_idx,: ], Y.iloc[trn_idx ]
    val_data, val_y = train_stack.iloc[ val_idx ,:  ], Y.iloc[val_idx]

    clf_stack =linear_model.LinearRegression()
    clf_stack.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_stack.predict(val_data)
    predictions += clf_stack.predict(test_stack) / 10
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_stack, Y)))

CV score: 0.10325199


In [12]:
len(oof_stack)

500000

In [13]:
y_hat =[]
for  y in oof_stack:
    if y>0.5:
        y_hat.append(1) 
    else :
        y_hat.append(0)

In [14]:
from sklearn.metrics import accuracy_score 
y = train['label'].values
accuracy_score( y, y_hat ) # 训练集合

0.859794

In [21]:
test=pd.read_csv( 'test_lightgbm.csv')
test['label'] = predictions
test[['sid'  , 'label']].to_csv('submit_probility.csv')
test['label']  = test['label'] .map( lambda x : 1 if x>0.5 else 0 )
y_hat = test [['sid' ,'label']].sort_values(by ='sid')['label'].values

test_ans = pd.read_csv("../test1_ans.csv")
y_true = test_ans.sort_values(by ='sid')['label'].values
test[['sid'  , 'label']].to_csv('submit.csv'  ,index =False )

In [17]:
accuracy_score( y_true , y_hat )

0.8510066666666667

In [18]:
predictions_xgb

0         0.218912
1         0.568260
2         0.069121
3         0.069065
4         0.896793
            ...   
149995    0.901183
149996    0.908581
149997    0.908655
149998    0.936764
149999    0.930452
Name: label, Length: 150000, dtype: float64

In [19]:
predictions_lgb

0         0.185634
1         0.655126
2         0.077574
3         0.058751
4         0.878399
            ...   
149995    0.914457
149996    0.954450
149997    0.862711
149998    0.958750
149999    0.950658
Name: label, Length: 150000, dtype: float64

In [20]:
predictions

array([0.20041622, 0.60789016, 0.06681428, ..., 0.89475164, 0.95322598,
       0.94603592])